In [55]:
import gym
import minerl
import logging
import numpy as np
import collections
from itertools import chain
%run DQNAgent.ipynb
logging.basicConfig(level=logging.DEBUG)

In [2]:
env = gym.make('MineRLTreechop-v0')
device = torch.device("cuda")

INFO:minerl.env.malmo.instance.be7ff2:Starting Minecraft process: ['C:\\Users\\liamo\\AppData\\Local\\Temp\\tmp4gikqpuk\\Minecraft\\launchClient.bat', '-port', '11501', '-env', '-runDir', 'C:\\Users\\liamo\\AppData\\Local\\Temp\\tmp4gikqpuk\\Minecraft\\run']
INFO:minerl.env.malmo.instance.be7ff2:Starting process watcher for process 31336 @ localhost:11501
DEBUG:minerl.env.malmo.instance.be7ff2:This mapping 'snapshot_20161220' was designed for MC 1.11! Use at your own peril.
DEBUG:minerl.env.malmo.instance.be7ff2:#################################################
DEBUG:minerl.env.malmo.instance.be7ff2:         ForgeGradle 2.2-SNAPSHOT-3966cea        
DEBUG:minerl.env.malmo.instance.be7ff2:  https://github.com/MinecraftForge/ForgeGradle  
DEBUG:minerl.env.malmo.instance.be7ff2:#################################################
DEBUG:minerl.env.malmo.instance.be7ff2:               Powered by MCP unknown               
DEBUG:minerl.env.malmo.instance.be7ff2:             http://modcoderpack.c

DEBUG:minerl.env.malmo.instance.be7ff2:[21:50:01] [main/WARN]: The coremod com.microsoft.Malmo.OverclockingPlugin does not have a MCVersion annotation, it may cause issues with this version of Minecraft
DEBUG:minerl.env.malmo.instance.be7ff2:2020-11-10 21:50:01,284 WARN Unable to instantiate org.fusesource.jansi.WindowsAnsiOutputStream
DEBUG:minerl.env.malmo.instance.be7ff2:2020-11-10 21:50:01,400 WARN Unable to instantiate org.fusesource.jansi.WindowsAnsiOutputStream
DEBUG:minerl.env.malmo.instance.be7ff2:[21:50:01] [main/INFO]: SpongePowered MIXIN Subsystem Version=0.7.5 Source=file:/C:/Users/liamo/.gradle/caches/modules-2/files-2.1/org.spongepowered/mixin/0.7.5-SNAPSHOT/c989878008c2c5ff1a7c6491c4103d8faa221d19/mixin-0.7.5-SNAPSHOT.jar Service=LaunchWrapper Env=CLIENT
DEBUG:minerl.env.malmo.instance.be7ff2:[21:50:01] [main/INFO]: Compatibility level set to JAVA_8
DEBUG:minerl.env.malmo.instance.be7ff2:[21:50:01] [main/INFO]: Calling tweak class net.minecraftforge.gradle.tweakers.Core

DEBUG:minerl.env.malmo.instance.be7ff2:[21:50:19] [Client thread/INFO]: Reloading ResourceManager: Default, FMLFileResourcePack:Forge Mod Loader, FMLFileResourcePack:Minecraft Forge, FMLFileResourcePack:Microsoft Malmo Mod
DEBUG:minerl.env.malmo.instance.be7ff2:[21:50:19] [Client thread/WARN]: ResourcePack: ignored non-lowercase namespace: MalmoMod in C:\Users\liamo\Anaconda3\envs\FallSemester\Lib\site-packages\minerl\env\Malmo\Minecraft\build\libs\MalmoMod-0.37.0.jar
DEBUG:minerl.env.malmo.instance.be7ff2:[21:50:19] [Client thread/WARN]: ResourcePack: ignored non-lowercase namespace: MalmoMod in C:\Users\liamo\Anaconda3\envs\FallSemester\Lib\site-packages\minerl\env\Malmo\Minecraft\build\libs\MalmoMod-0.37.0.jar
DEBUG:minerl.env.malmo.instance.be7ff2:[21:50:19] [Client thread/WARN]: ResourcePack: ignored non-lowercase namespace: MalmoMod in C:\Users\liamo\Anaconda3\envs\FallSemester\Lib\site-packages\minerl\env\Malmo\Minecraft\build\libs\MalmoMod-0.37.0.jar
DEBUG:minerl.env.malmo.inst

In [3]:
print(env.observation_space.sample()["pov"].shape) #An RGB image observation of the agent’s first-person perspective
print(env.action_space.sample())      #https://minerl.io/docs/environments/index.html#action-space

(64, 64, 3)
OrderedDict([('attack', array(1)), ('back', array(0)), ('camera', array([144.52388, 135.6594 ], dtype=float32)), ('forward', array(0)), ('jump', array(0)), ('left', array(0)), ('right', array(1)), ('sneak', array(1)), ('sprint', array(0))])


In [30]:
def train(env, agent, config):
    num_episodes = config["NUM_EPISODES"]

    episodic_rewards = [0] * num_episodes
    episodic_epsilon = [0] * num_episodes
    episodic_moves = [0] * num_episodes

    for episode in range(num_episodes):
        start_state = env.reset()
        done = False

        # MDP Loop
        steps = 0
        pov = start_state["pov"].reshape(3,64,64)
        while not done:
            pov_tensor = torch.from_numpy(pov).float().to(device).unsqueeze(0)
            pov_tensor.shape
            
            action = agent.step(pov_tensor)
            steps = steps + 1
            
            obs, reward, done, info = env.step(action)
            
            next_pov = obs["pov"].reshape(3,64,64)
            if done:
                next_pov_tensor = None
            else:
                next_pov_tensor = torch.from_numpy(next_pov).float().to(device).unsqueeze(0)
            
            flat_actions = np.hstack(list(action.values()))
            action_tensor = torch.tensor(flat_actions, device=device)
            
            reward_tensor = torch.tensor([reward], device=device)
            
            agent.memory.store_transition(Transition(pov_tensor, action_tensor, next_pov_tensor, reward_tensor))
            
            if steps % config["EVAL_INTERVAL"] == 0:
                agent.evaluate(pov_tensor, action, next_pov_tensor, reward)
            
            pov = next_pov
                    
            episodic_rewards[episode] += reward
            episodic_moves[episode] += 1
            
        if episode % config["SYNC_INTERVAL"] == 0:
            agent.agent_sync_networks()
        episodic_epsilon[episode] = agent.epsilon
        agent.decay_epsilon()
        
    return episodic_rewards, episodic_epsilon, episodic_moves

In [56]:
agent_config = {
    "EPS": .99,
    "EPS_DECAY": .995,
    "GAMMA": 1,
    "MEMORY_CAPACITY": 100000,
    "MEMORY_ALPHA": .7,
    "MEMORY_BETA": .4,
    "MEMORY_EPSILON": .001,
    "BATCH_SIZE": 64
}
agent = DQNAgent(env, agent_config)

In [57]:
train_config = {
    "EVAL_INTERVAL": 10,
    "SYNC_INTERVAL": 5,
    "NUM_EPISODES": 10
}
rew, ep, moves = train(env, agent, train_config)

DEBUG:minerl.env.malmo.instance.be7ff2:[22:37:51] [Client thread/INFO]: [STDOUT]: Unpausing
DEBUG:minerl.env.core:Sending mission init!
DEBUG:minerl.env.malmo.instance.be7ff2:[22:37:51] [Client thread/INFO]: [STDOUT]: CLIENT request state: MISSION_ABORTED
DEBUG:minerl.env.malmo.instance.be7ff2:[22:37:51] [Client thread/INFO]: [STDOUT]: CLIENT enter state: MISSION_ABORTED
DEBUG:minerl.env.malmo.instance.be7ff2:[22:37:51] [Client thread/INFO]: [STDOUT]: inform the agent
DEBUG:minerl.env.malmo.instance.be7ff2:[22:37:51] [Client thread/INFO]: [STDOUT]: CLIENT request state: DORMANT
DEBUG:minerl.env.malmo.instance.be7ff2:[22:37:51] [Client thread/INFO]: [STDOUT]: CLIENT enter state: DORMANT
DEBUG:minerl.env.malmo.instance.be7ff2:[22:37:51] [Client thread/INFO]: [STDOUT]: JETTISONING 0 COMMANDS
DEBUG:minerl.env.malmo.instance.be7ff2:[22:37:51] [EnvServerSocketHandler/INFO]: [STDOUT]: Received from epicor-web-local:<MissionInit xmlns="http://ProjectMalmo.microsoft.com" xmlns:xsi="http://www.w

DEBUG:minerl.env.malmo.instance.be7ff2:[22:38:04] [Server thread/INFO]: Loading dimension 1 (TEMP_0_81aff9bf-97f6-48ef-91ff-b44808877509) (net.minecraft.server.integrated.IntegratedServer@7fa2e821)
DEBUG:minerl.env.malmo.instance.be7ff2:[22:38:04] [Server thread/INFO]: Loading dimension -1 (TEMP_0_81aff9bf-97f6-48ef-91ff-b44808877509) (net.minecraft.server.integrated.IntegratedServer@7fa2e821)
DEBUG:minerl.env.malmo.instance.be7ff2:[22:38:04] [Server thread/INFO]: Preparing start region for level 0
DEBUG:minerl.env.malmo.instance.be7ff2:[22:38:05] [Server thread/INFO]: Preparing spawn area: 17%
DEBUG:minerl.env.malmo.instance.be7ff2:[22:38:06] [Server thread/INFO]: Preparing spawn area: 34%
DEBUG:minerl.env.malmo.instance.be7ff2:[22:38:07] [Server thread/INFO]: Preparing spawn area: 51%
DEBUG:minerl.env.malmo.instance.be7ff2:[22:38:08] [Server thread/INFO]: Preparing spawn area: 68%
DEBUG:minerl.env.malmo.instance.be7ff2:[22:38:09] [Server thread/INFO]: Preparing spawn area: 85%
DEBUG:

tensor(4, device='cuda:0')
tensor(4, device='cuda:0')


DEBUG:minerl.env.malmo.instance.be7ff2:[22:38:12] [Client thread/INFO]: [CHAT] §l397...
DEBUG:minerl.env.malmo.instance.be7ff2:[22:38:12] [Client thread/INFO]: [CHAT] §l396...


tensor(4, device='cuda:0')
tensor(4, device='cuda:0')
tensor([[ 7.9062e-01,  8.2661e-01, -2.6753e-01,  5.0976e-01,  3.0731e+00,
         -1.2210e+00, -1.0329e+00,  1.5302e+00,  1.1551e+00],
        [-3.6890e-01,  3.3513e-01,  4.2782e-01, -1.0967e-01,  1.9024e+00,
         -7.4051e-01, -9.3901e-01,  9.4145e-01,  1.1710e+00],
        [ 1.1928e-01,  5.6213e-01,  3.1888e-01,  9.0530e-02,  2.5658e+00,
         -1.1883e+00, -1.0681e+00,  1.3530e+00,  1.4443e+00],
        [ 3.7472e-01,  1.0879e+00, -1.5251e-01, -1.6429e-01,  3.8592e+00,
         -2.0145e+00, -1.4316e+00,  1.7676e+00,  1.9260e+00],
        [ 2.9134e-01,  4.2350e-01,  1.9500e-01,  5.1139e-02,  2.0623e+00,
         -8.9180e-01, -8.3122e-01,  1.0887e+00,  9.3844e-01],
        [-6.8053e-01,  1.6209e-01,  9.1234e-01, -9.5638e-02,  1.7390e+00,
         -7.5312e-01, -5.4473e-01,  2.6143e-01,  7.9882e-01],
        [ 6.0481e-01,  1.8227e-01,  2.0662e-01,  2.7650e-01,  3.5836e+00,
         -1.4553e+00, -1.5555e+00,  2.1175e+00,  2.3502e

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)